## Code for Straight Run of Fixed Income Dashboard Update

#### _Step 1: Libraries_

In [3]:
import pandas as pd
import numpy as np
import datetime
import urllib.request
from urllib.error import URLError
import os
import slate3k as slate
import re
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_table

#### _Step 2: Inputs_

**Input latest GDP growth if Applicable (Quarterly Release, ~1 month and 1 week after quarter end)**

In [21]:
quarter_input='Q2'
year_input='2019 - 2020'
growth_input=-16.5

**Input latest Inflation if Applicable (Monthly Release, ~ 1 week after month end)**

In [24]:
year_input=2020
month_input=7
inflation_input=2.7

**Input latest dates for BVAL rates update**

In [27]:
Start_Date="2020-08-26"
End_Date="2020-09-01"

#### _Step 3: Update Databases with Inputs_

In [23]:
ph_gdp=pd.read_csv("PH_GDP.csv")
new_gdp=pd.DataFrame([quarter_input,str(year_input),growth_input]).transpose()
new_gdp.columns=['quarter','year','GDP']
updated_gdp=ph_gdp.append(new_gdp)
updated_gdp.to_csv("PH_GDP.csv",index=False)

In [26]:
infl=pd.read_csv("PH_inflation_historical.csv")
new_infl=pd.DataFrame([int(year_input),int(month_input),inflation_input]).transpose()
new_infl.columns=['Year','Month','Inflation']
updated_infl=infl.append(new_infl)
cols=['Year','Month']
updated_infl[cols]=updated_infl[cols].applymap(np.int64)
updated_infl.to_csv("PH_inflation_historical.csv",index=False)

In [28]:
# Function to convert string to datetime
#Source: https://www.geeksforgeeks.org/python-convert-string-to-datetime-and-vice-versa/
def convert(date_time): 
    format = '%Y-%m-%d' # The format 
    datetime_str = datetime.datetime.strptime(date_time, format) 
    return datetime_str

#Convert DateType
Start_Date=convert(Start_Date)
End_Date=convert(End_Date)
DateDiff=(End_Date-Start_Date).days

# Dates/Files to extract
fname_format_start='https://www.pds.com.ph/wp-content/uploads/'
fname_format_end= 'PHP-BVAL-Reference-Rates-Benchmark-Tenors_'
DateList=[]
for i in range(DateDiff+1):
    new_date=Start_Date + datetime.timedelta(i)
    str_date=str(new_date.date()).split('-')
    f_name=fname_format_start+str_date[0]+"/"+str_date[1]+"/"+fname_format_end+str(new_date.date())+'.pdf'
    DateList.append(f_name)

In [30]:
# Download if files exist, otherwise PASS
pattern=re.compile("Benchmark-Tenors_(\S+).pdf")
myPath='C:/Users/j_bia/Documents/projects/FixedIncomeDashboardCreation/LatestBVALrates/'

for link in DateList:
    #filename
    name_date=pattern.search(link.split('/')[-1]).group(1)
    fullfilename = os.path.join(myPath, name_date+'.pdf')
    try:
        urllib.request.urlretrieve(link, fullfilename)
    except URLError:
        pass

In [31]:
# Compile downloaded files in a dataframe containing latest BVAL rates
folder_name=r"C:\Users\j_bia\Documents\projects\FixedIncomeDashboardCreation\LatestBVALrates"
f_names=os.listdir(folder_name)

pattern=re.compile("(\S+).pdf")
BVALrates=pd.DataFrame([])

for filename in f_names:
    #get the date for the BVAL rate to be extracted
    name_date=pattern.search(filename).group(1)
    
    #extract rates, clean using Regex
    with open(folder_name+'\\'+ filename, 'rb') as f:
        extracted_text = slate.PDF(f)
        
    subbed=re.sub(r'\\n\\n', ',', str(extracted_text))
    subbed=re.sub(r'\[\'','',subbed)
    subbed=re.sub(r'\\x0c\'\]','',subbed)
    list_subbed=subbed.split(",")
       
    BVAL_today=pd.DataFrame(list_subbed)[15:27]
    BVAL_today.columns=[name_date]
    BVAL_today.reset_index(inplace=True)
    BVAL_today=BVAL_today[[name_date]].transpose()
        
    BVALrates=BVALrates.append(BVAL_today)

BVALrates.reset_index(inplace=True)
BVALrates.columns=['DATE','1M','3M','6M','1Y','2Y','3Y','4Y','5Y','7Y','10Y','20Y','25Y']
# After loading, manually transfer the pdf files in "LatestBVALrates" to "BVALratesPDF"

In [57]:
BVALrates

,DATE,1M,3M,6M,1Y,2Y,3Y,4Y,5Y,7Y,10Y,20Y,25Y
0,2020-08-26,1.120,1.194,1.429,1.790,2.072,2.224,2.364,2.490,2.659,2.785,3.484,3.654
1,2020-08-27,1.130,1.201,1.437,1.802,2.102,2.263,2.408,2.531,2.700,2.778,3.520,3.647
2,2020-08-28,1.132,1.203,1.440,1.815,2.118,2.284,2.426,2.544,2.704,2.797,3.520,3.630
3,2020-09-01,1.123,1.199,1.449,1.813,2.150,2.316,2.456,2.566,2.715,2.803,3.610,3.791


In [34]:
# Append the updated rates to historical database
historical=pd.read_csv("PHreferencerates.csv")
updated=pd.concat([historical,BVALrates],ignore_index=True)
updated.set_index('DATE',inplace=True)
updated.to_csv("PHreferencerates.csv")

#### _Step 4: Computing Z-scores for Term Spreads_

In [4]:
df = pd.read_csv (r'C:\Users\j_bia\Documents\projects\FixedIncomeDashboardCreation\PHreferencerates.csv')
df.set_index("DATE",inplace=True)

#Create Spreads
Spread_20Y10Y=df['20Y']-df['10Y']
Spread_10Y7Y=df['10Y']-df['7Y']
Spread_10Y5Y=df['10Y']-df['5Y']
Spread_10Y2Y=df['10Y']-df['2Y']
Spread_7Y5Y=df['7Y']-df['5Y']
Spread_7Y2Y=df['7Y']-df['2Y']
Spread_5Y2Y=df['5Y']-df['2Y']
Spread_2Y1Y=df['2Y']-df['1Y']

#Combine Spreads to dataframe
Spreads=pd.concat([Spread_20Y10Y,Spread_10Y7Y,Spread_10Y5Y,Spread_10Y2Y,Spread_7Y5Y,Spread_7Y2Y,Spread_5Y2Y,Spread_2Y1Y],axis=1)

#rename the columns
Spreads.columns=['20-10','10-7','10-5','10-2','7-5','7-2','5-2','2-1']

#1 YEAR Z
#Get the rolling average spread
spread_ave_1year=Spreads.rolling(260).mean()[259:]
# Get rolling standard deviations
spread_vol_1year=Spreads.rolling(260).std()[259:]
# Get same size for the "X" of (X-mu)/sigma
spreads=Spreads[259:]
# Get difference (X-mu)
diff_1year=spreads.subtract(spread_ave_1year)
# Divide by vol to get Z
spread_Z_1year=diff_1year.divide(spread_vol_1year)
spread_Z_1year=spread_Z_1year.round(decimals=2)

#5 YEAR Z
# Get the rolling average spread
spread_ave_5year=Spreads.rolling(260*5).mean()[260*5-1:]
# Get rolling standard deviations
spread_vol_5year=Spreads.rolling(260*5).std()[260*5-1:]
# Get same size
spreads_5year=Spreads[260*5-1:]
# Get difference (X-mu)
diff_5year=spreads_5year.subtract(spread_ave_5year)
# Divide by vol to get Z
spread_Z_5year=diff_5year.divide(spread_vol_5year)
spread_Z_5year=spread_Z_5year.round(decimals=2)


# Get Latest Z-scores
latest_1yr_Z=spread_Z_1year.tail(1).transpose()
latest_1yr_Z.columns=["1-year Z-score"]
latest_5yr_Z=spread_Z_5year.tail(1).transpose()
latest_5yr_Z.columns=["5-year Z-score"]
latest_Z=pd.concat([latest_1yr_Z,latest_5yr_Z],axis=1)
latest_Z.to_csv("latest_Z_termspread.csv")

#### _Step 5: Computing Z-scores for Real Yields_

In [5]:
inflation=pd.read_csv("PH_inflation_historical.csv")
rates=pd.read_csv("PHreferencerates.csv")

# convert DATE to datetime format
# only have inflation 2013 onwards (since base=2012)
# get only rates 2013 onwards as well
rates['DATE']=pd.to_datetime(rates['DATE'])
rates=rates[rates.DATE>'2013-01-01']

# add DATE column to inflation
inflation['DATE']=inflation.apply(lambda row: datetime.datetime(int(row.Year),int(row.Month),1),axis=1)

# Merge on DATE
combined=pd.merge_asof(rates,inflation,on='DATE')

# Create the real yields dataframe
tenors=['1M','3M','6M','1Y','2Y','3Y','4Y','5Y','7Y','10Y','20Y']
realyields=pd.DataFrame([combined[x]-combined['Inflation'] for x in tenors]).transpose()
realyields.columns=tenors
realyields['DATE']=combined['DATE']
realyields.set_index('DATE',inplace=True)

#1 year Z-score
#Get the rolling average real yield
realyield_ave_1year=realyields.rolling(260).mean()[259:]
#Get the rolling std of real yields
realyield_vol_1year=realyields.rolling(260).std()[259:]
# Get same size
realyields_1year=realyields[259:]
# Get difference (X-mu)
diff_realyield_1year=realyields_1year.subtract(realyield_ave_1year)
# Divide by vol to get Z
realyield_Z_1year=diff_realyield_1year.divide(realyield_vol_1year)
realyield_Z_1year=realyield_Z_1year.round(decimals=2)

#5 year Z-score
#Get the rolling average real yield
realyield_ave_5year=realyields.rolling(260*5).mean()[260*5-1:]
#Get the rolling std of real yields
realyield_vol_5year=realyields.rolling(260*5).std()[260*5-1:]
# Get same size
realyields_5year=realyields[260*5-1:]
# Get difference (X-mu)
diff_realyield_5year=realyields_5year.subtract(realyield_ave_5year)
# Divide by vol to get Z
realyield_Z_5year=diff_realyield_5year.divide(realyield_vol_5year)
realyield_Z_5year=realyield_Z_5year.round(decimals=2)

latest1yr_Z_realyield=realyield_Z_1year.tail(1).transpose()
latest1yr_Z_realyield.columns=['Real Yield 1-year Z']
latest5yr_Z_realyield=realyield_Z_5year.tail(1).transpose()
latest5yr_Z_realyield.columns=['Real Yield 5-year Z']
latestZ_realyields=pd.concat([latest1yr_Z_realyield,latest5yr_Z_realyield],axis=1)
latestZ_realyields.to_csv("latestZ_realyields.csv")

#### _Step 6: Dashboard Creation using DASH_

In [6]:
gdp=pd.read_csv("PH_GDP.csv")
def convert_to_date(quarter,year_range):
    #get last 4 characters of year_range
    year=int(year_range[-4:])
    #set month and day for quarters
    if quarter=='Q1':
        month=3
        day=31
    elif quarter=='Q2':
        month=6
        day=30
    elif quarter=='Q3':
        month=9
        day=30
    else:
        month=12
        day=31
   
    return datetime.datetime(year,month,day)
gdp['Date']=pd.DataFrame([convert_to_date(gdp['quarter'][x],gdp['year'][x]) for x in range(0,len(gdp))])

inflation=pd.read_csv("PH_inflation_historical.csv")
inflation['Date']=inflation.apply(lambda row: datetime.datetime(int(row.Year),int(row.Month),1),axis=1)

realyields=pd.read_csv("latestZ_realyields.csv")
realyields.columns=['Tenor','Real Yield 1-year Z','Real Yield 5-year Z']
termspreads=pd.read_csv('latest_Z_termspread.csv')
termspreads.columns=['Spread','1-year Z-score','5-year Z-score']

df = pd.read_csv(r'C:\Users\j_bia\Documents\projects\FixedIncomeDashboardCreation\PHreferencerates.csv')

In [7]:
app = JupyterDash(__name__)
available_tenors = df.columns[1:]

#create font_color for Term Spreads
font_color1=['rgb(40,40,40)']
labels=list(termspreads['Spread'])
Zspreadsvals=[list(termspreads['1-year Z-score']),list(termspreads['5-year Z-score'])]
for k in range(len(Zspreadsvals)):
    for v in Zspreadsvals[k]:
        if v<=0:
            mix='rgb(255,0,0)'
        elif v< 1:
            mix='rgb(50,50,50)'
        else:
            mix='rgb(35,135,50)'
        font_color1.append(mix)
font_color1=[font_color1[0],font_color1[1:9],font_color1[9:]]

#create font_color for Real Yields
font_color2 = ['rgb(40,40,40)'] +  [['rgb(255,0,0)' if v <= 0 else 'rgb(10,10,10)' 
                                     for v in realyields[k]] for k in realyields.columns[1:]]


# Create Term Spreads Z-table
fig0=go.Figure(data=[go.Table(header=dict(values=list(termspreads.columns),
                                          fill_color='#FBDDF0',align='center'),
                           cells=dict(values=[labels,Zspreadsvals[0],Zspreadsvals[1]],
                                      fill_color='#F2F2F2',align='center', font=dict(color=font_color1)))])


# Create Real Yields Z-table
fig1=go.Figure(data=[go.Table(header=dict(values=list(realyields.columns),fill_color='#FBDDF0',align='center'),
                             cells=dict(values=[realyields['Tenor'],realyields['Real Yield 1-year Z'],
                                                realyields['Real Yield 5-year Z']],
                                        fill_color='#F2F2F2',align='center',
                                       font=dict(color=font_color2)))])

app.layout = html.Div([

        html.Div([
            dcc.Graph(
            id='PH_inflation',
            figure=px.line(inflation,x='Date',y='Inflation',title='Philippine Inflation'))],
        style={'width': '50%', 'display': 'inline-block'}),
    
        html.Div([
            dcc.Graph(
            id='PH GDP',
            figure=px.line(gdp,x='Date',y='GDP',title='Philippine GDP Growth'))],
        style={'width': '50%', 'display': 'inline-block'}),
    
        html.Div([
            dcc.Dropdown(
                id='yield_curve',
                options=[{'label': i, 'value': i} for i in available_tenors],
                value='10Y',
                multi=True
            ),
            dcc.Graph(id='line_chart')],),
        
        html.Div([
            dcc.Graph(
            id='Spreads_Zscore',
            figure=fig0)],
        style={'width': '50%', 'display': 'inline-block'}),
    
        html.Div([
            dcc.Graph(
            id='RealYield_Zscore',
            figure=fig1)],
        style={'width': '50%', 'display': 'inline-block'})
    
            
])

@app.callback(
    Output('line_chart', 'figure'),
    [Input('yield_curve', 'value')])

def update_graph(yaxis_column_name):
    fig = px.line(df, x='DATE',
                     y=yaxis_column_name, title="PH yield curve")
    fig.update_layout(margin={'l': 40, 'b': 40, 't': 30, 'r': 30}, hovermode='closest')    
    return fig

app.run_server(mode='external',port=8080)

Dash app running on http://127.0.0.1:8080/


#### Optional Step: Running on Datapane

In [8]:
import datapane as dp

In [16]:
r=dp.Report(
    dp.Markdown("Fixed Income Dashboard"),
    dp.Plot(px.line(gdp,x='Date',y='GDP',title='Philippine GDP Growth')),
    dp.Plot(px.line(inflation,x='Date',y='Inflation',title='Philippine Inflation')),
    dp.Plot(fig0),
    dp.Plot(fig1)
)
r.publish(name="PH_GDP_Inflation_Data",visibility='PUBLIC')

Publishing report and associated data - please wait..
Report successfully published at https://datapane.com/bianca0317/reports/PH_GDP_Inflation_Data/
